# Processamento de Dados em Larga Escala com Spark

## Informação Básica
- **Título do Projeto**: Processamento de Dados em Larga Escala com Spark
- **Alunos**:
  - João Carneiro, Nº 50938
  - Eduardo Abrantes, Nº 50391

## Contribuição

| Aluno        | Tarefa realizada                                                                 | Horas estimadas |
|--------------|----------------------------------------------------------------------------------|-----------------|
| João Carneiro   | ...  | 7h             |
| Eduardo Abrantes  | ...        | 7h             |

## Background e Motivação

- Este projeto tem como objetivo ...

# Pré-Requisitos

## Instalação dos Dados
Antes de iniciar o projeto, é necessário realizar o download e preparação dos Dados Meteorológicos do GHCN-Daily. Siga os passos abaixo:

## Setup de Dependências
Para a realização deste projeto, foram utilizadas bibliotecas fundamentais do Python para Ciência de Dados. A instalação das dependências foi realizada através do comando:

In [23]:
!pip install wikiextractor

#### Descrição das Bibliotecas

- wikiextractor: ...

- ...

## Extração do Dump da Wikipedia

...

O seguinte código foi utilizado para esta operação inicial:

In [ ]:
!python -m wikiextractor.WikiExtractor "/home/jovyan/work/proj-three-data-science/data/enwiki-latest-pages-articles.xml.bz2" -o "/home/jovyan/work/proj-three-data-science/data/wikipedia-dump/text" --no-templates

## Leitura dos Ficheiros com PySpark

In [11]:
from pyspark.sql import SparkSession
import time

# 'spark.driver.maxResultSize' está englobado dentro do 'spark.driver.memory', onde o anterior é o tamanho máximo que os resultados podem ocupar em memória, e o posterior está a acontar com o overhead dos objetos na JVM
# 'spark.executor.memory', é a quantidade de memórioa reservada para cada processo em cada node do cluster, neste caso o node do cluster executa na mesma máquina.
# 'spark.network.timeout' e 'spark.executor.heartbeatInterval' são usados para verificar se a conexão foi perdida, mas com grandes datasets tem de ser aumentada
# Se tiver a usar WSL, poderá ser necessário mudar a quantidaade de RAM disponivel, pois por default apenas pode usar até metade da RAM do PC (https://learn.microsoft.com/en-us/windows/wsl/wsl-config#example-wslconfig-file)

spark = SparkSession.builder \
    .appName("WikipediaDump") \
    .master("local[*]") \
    .config("spark.driver.memory", "12g") \
    .config("spark.driver.maxResultSize", "8g") \
    .config("spark.executor.memory", "12g") \
    .config("spark.network.timeout", "800s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .getOrCreate()


sc = spark.sparkContext

# Lê todos os ficheiros de texto extraídos
#rdd = sc.wholeTextFiles("/home/jovyan/work/proj-three-data-science/data/wikipedia-dump/text/A*/*")
rdd = sc.wholeTextFiles("/home/jovyan/work/proj-three-data-science/data/wikipedia-dump/text/*/*")

# Exercícios

## Exercício 1 ...

In [12]:
import re

def extract_pages(file_content):
    # Tupula no formato de (filename, text)
    text = file_content[1]
    docs = re.findall(r"<doc(.*?)</doc>", text, re.DOTALL)
    result = []

    for doc in docs:
        # Extract url e titulo de dentro da tag
        header = re.search(r'url="(.*?)".*?title="(.*?)">', doc)
        if not header:
            continue
        url = header.group(1)
        title = header.group(2)

        # Extrair o connteudo (após a tag)
        content_match = re.search(r'">(.*?)$', doc, re.DOTALL)
        if not content_match:
            continue
        content = content_match.group(1).strip()

        #Ignorar se não tiver realmente conteudo
        lines = content.splitlines()
        if len(lines) < 3 or not lines[2].strip():
            continue

        # \n join pois o splitlines remove o \n
        filtered_content = "\n".join(lines[2:]).strip()

        result.append((url, title, filtered_content))

    return result

In [13]:
rdd = rdd.flatMap(extract_pages)

## Exercício 2

### Exercício 2.1 - Resultado dos primeiros dois Elementos do RDD

Neste exercício apenas se pretende mostrar os primeiros dois elementos por isso recorremos à Ação RDD `take` do PySpark 
que apenas retorna um determinado número de elementos que neste caso foram dois.

In [4]:
start = time.perf_counter()

ex2_1 = rdd.take(2)
print(ex2_1)

end = time.perf_counter()
print(f"Demorou {round(end - start, 2)} segundos a processar.")

[('https://en.wikipedia.org/wiki?curid=12', 'Anarchism', 'Anarchism is a political philosophy and movement that seeks to abolish all institutions that perpetuate authority, coercion, or hierarchy, primarily targeting the state and capitalism. Anarchism advocates for the replacement of the state with stateless societies and voluntary free associations. A historically left-wing movement, anarchism is usually described as the libertarian wing of the socialist movement (libertarian socialism).\nAlthough traces of anarchist ideas are found all throughout history, modern anarchism emerged from the Enlightenment. During the latter half of the 19th and the first decades of the 20th century, the anarchist movement flourished in most parts of the world and had a significant role in workers\' struggles for emancipation. Various anarchist schools of thought formed during this period. Anarchists have taken part in several revolutions, most notably in the Paris Commune, the Russian Civil War and the

### Exercício 2.2 - Total de Elementos do RDD

Neste exercício pretendia-se realizar uma contagem do número de elementos que compôem o RDD, para isso recorremos à Ação 
RDD `count` do PySpark que contabiliza o número de elementos totais.

In [5]:
start = time.perf_counter()

ex2_2 = rdd.count()
print(f"Número de elementos do RDD: {ex2_2}")

end = time.perf_counter()
print(f"Demorou {round(end - start, 2)} segundos a processar.")

Número de elementos do RDD: 6885369
Demorou 274.42 segundos a processar.


### Exercício 2.3 - Número de Caracteres de cada Página Web

Neste exercício o objetivo é contabilizar os caracteres de cada uma das páginas Webs do RDD, para isso primeiro usamos uma 
transformação RDD chamada de `map` em que separa o `url` e o `title` de forma a identificar a página Web que se está a referir 
e o `content` é feita a contagem dos caracteres dessa página Web. 
Depois disso devido às transformações serem `Lazy` é necessário executar uma ação RDD que neste caso decidimos usar o `collect`
para executar a transformação e também conseguir aceder aos resultados com essa ação. Apesar de não ser recomendado
decidimos usar a ação `collect` devido a ... 
Por fim devido ao grande volume de dados apenas decidimos mostrar os primeiros 5 elementos da lista como exemplo.

In [6]:
start = time.perf_counter()

contagem_caracteres_rdd = rdd.map(lambda x: (x[0], x[1], len(x[2])))
all_contagem_caracteres = contagem_caracteres_rdd.collect()
resultado_parcial = all_contagem_caracteres[:5]

for url, title, contagem_caracteres in resultado_parcial:
    print(f"URL: {url}\nTítulo: {title}\nNº caracteres: {contagem_caracteres}\n")



end = time.perf_counter()
print(f"Demorou {round(end - start, 2)} segundos a processar.")


URL: https://en.wikipedia.org/wiki?curid=12
Título: Anarchism
Nº caracteres: 43674

URL: https://en.wikipedia.org/wiki?curid=39
Título: Albedo
Nº caracteres: 23857

URL: https://en.wikipedia.org/wiki?curid=290
Título: A
Nº caracteres: 6453

URL: https://en.wikipedia.org/wiki?curid=303
Título: Alabama
Nº caracteres: 77446

URL: https://en.wikipedia.org/wiki?curid=305
Título: Achilles
Nº caracteres: 36523

Demorou 237.77 segundos a processar.


### Exercício 2.4 - Total de Caracteres de todas as Páginas Web

Neste exercício também tem como objetivo contabilizar os caracteres das páginas Webs do RDD e somar cada um dando o total 
como resultado. Usamos a transformação RDD `map` em que apenas calcula o tamanho do `content` de cada um dos elementos.
Depois como já expliquei anteriormente devido às transformações serem `Lazy` é necessário chamar uma ação que no caso foi
a `sum` mas também podia ser chamada a ação `reduce` e teria o mesmo efeito, essas duas ações juntam todos os valores e
somam.

In [8]:
start = time.perf_counter()

total_caracteres = rdd.map(lambda x: len(x[2])).sum()
print("Total de caracteres:", total_caracteres)

end = time.perf_counter()
print(f"Demorou {round(end - start, 2)} segundos a processar.")

Total de caracteres: 16973563947
Demorou 254.57 segundos a processar.


### Exercício 2.5 - Conjunto das Páginas de Celebridades

Neste exercício tem como objetivo de determinar as páginas de celebridades que estejam no RDD, para isso foi usado um
`regex` feito pelo ChatGPT em que deteta se existem datas de nascimento. Filtramos as páginas do RDD para obter as páginas 
que indicam celebridades, com base na data de nascimento nos primeiros 250 caracteres do `content`. 
Também foi usada a ação RDD `collect` para executar a transformação anterior e conseguir aceder aos resultados.
Por fim devido ao grande volume de dados apenas decidimos mostrar os primeiros 5 elementos da lista como exemplo.

In [15]:
import re

start = time.perf_counter()

# re.compile para poder usar o .search() ou .match()
# r string para não dar "escape" aos special characters
# Regex feito pelo GPT e testado no site https://regex101.com/ (selecionar modo python)
pattern = r"(\d{1,2})\s+(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December)\s+(\d{4})"
regex = re.compile(pattern, re.IGNORECASE)

rdd_celebridades = rdd.filter(lambda page: bool(regex.search(page[2][:250])))

celebridades = rdd_celebridades.collect()

resultado_parcial = celebridades[:5]

for url, title, content in resultado_parcial:
    print(f"URL: {url}\nTítulo: {title}\nConteudo: {content}\n")


end = time.perf_counter()
print(f"Demorou {round(end - start, 2)} segundos a processar.")

URL: https://en.wikipedia.org/wiki?curid=340
Título: Alain Connes
Conteudo: Alain Connes (; born 1 April 1947) is a French mathematician, known for his contributions to the study of operator algebras and noncommutative geometry. He was a professor at the , , Ohio State University and Vanderbilt University. He was awarded the Fields Medal in 1982.
Career.
Alain Connes attended high school at in Marseille, and was then a student of the classes préparatoires in . Between 1966 and 1970 he studied at École normale supérieure in Paris, and in 1973 he obtained a PhD from Pierre and Marie Curie University, under the supervision of Jacques Dixmier.
From 1970 to 1974 he was research fellow at the French National Centre for Scientific Research and during 1975 he held a visiting position at Queen's University at Kingston in Canada.
In 1976 he returned to France and worked as professor at Pierre and Marie Curie University until 1980 and at CNRS between 1981 and 1984. Moreover, since 1979 he holds t

## Exercício 3 - Criação de um DataFrame com as Informações das Celebridades

Neste exercício reaproveitando o exercício anterior, filtramos as páginas do RDD para obter as páginas que indicam celebridades, com base na data de nascimento nos primeiros 250 caracteres do `content`. Em segundo, os resultados foram convertidos para um DataFrame Spark com as colunas `url`, `title` e `content`, e por fim utilizamos o método `show` para mostrar as 5 primeiras linhas do DataFrame.

In [25]:
# Reaproveitamento do ex anterior
pattern = r"(\d{1,2})\s+(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December)\s+(\d{4})"
regex = re.compile(pattern, re.IGNORECASE)

celebridades_df = (
    rdd.filter(lambda page: bool(regex.search(page[2][:250])))
             .toDF(["url", "title", "content"])
)

celebridades_df.show(5)

+--------------------+-------------------+--------------------+
|                 url|              title|             content|
+--------------------+-------------------+--------------------+
|https://en.wikipe...|       Alain Connes|Alain Connes (; b...|
|https://en.wikipe...|        Animal Farm|Animal Farm is a ...|
|https://en.wikipe...|      Aldous Huxley|Aldous Leonard Hu...|
|https://en.wikipe...|   Andrei Tarkovsky|Andrei Arsenyevic...|
|https://en.wikipe...|Arthur Schopenhauer|Arthur Schopenhau...|
+--------------------+-------------------+--------------------+
only showing top 5 rows



## Exercício 4 - Guardar o resultado completo em um ficheiro `.csv`

Neste exercício para a criação do ficheiro `.csv` usámos o DataFrame Spark que foi criado no exercício anterior com as informações das celebridades e também foi medido neste exercício o tempo de execução.

In [20]:
start = time.perf_counter()

# Ficheiro é guardado dentro da pasta junto de outros ficheiros para verificar a validade dele (.crc)
# coalesce(1) é para ele fazer apenas 1 parte em vez de várias
celebridades_df.coalesce(1).write.csv("/home/jovyan/work/celebridades_csv", header=True)

end = time.perf_counter()
print(f"Demorou {round(end - start, 2)} segundos a processar.")

Demorou 484.8 segundos a processar.


# Declaração de Integridade

> Eu, João Carneiro, estudante com o número de inscrição 50938 do 1º Ciclo em Informática Web, Móvel e na Nuvem da Universidade da Beira Interior, declaro ter desenvolvido o presente trabalho e elaborado o presente texto em total consonância com o Código de Integridade da Universidade da Beira Interior. Mais concretamente afirmo não ter incorrido em qualquer das variedades de Fraude Académica, e que aqui declaro conhecer, que em particular atendi à exigida referenciação de frases, extratos, imagens e outras formas de trabalho intelectual, e assumindo assim na íntegra as responsabilidades da autoria


> Eu, Eduardo Abrantes, estudante com o número de inscrição 50391 do 1º Ciclo em Informática Web, Móvel e na Nuvem da Universidade da Beira Interior, declaro ter desenvolvido o presente trabalho e elaborado o presente texto em total consonância com o Código de Integridade da Universidade da Beira Interior. Mais concretamente afirmo não ter incorrido em qualquer das variedades de Fraude Académica, e que aqui declaro conhecer, que em particular atendi à exigida referenciação de frases, extratos, imagens e outras formas de trabalho intelectual, e assumindo assim na íntegra as responsabilidades da autoria

Universidade da Beira Interior, Covilhã  
02/06/2025
